# `Itr` examples

All the examples here make use of the generators/iterators defined next. One is open-ended and the other (very probably) not.

### Fibonacci Generator and Iterator

Two implementations of Fibonacci sequences: a generator and an iterator class.


In [1]:
import contextlib
from collections.abc import Generator, Iterator
from typing import Self

from itrx import Itr

In [2]:
def fibonacci() -> Generator[int, None, None]:
    "Generator implementation of Fibonacci"
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b


class Fibonacci(Iterator[int]):
    "Iterator implementation of Fibonacci"

    def __init__(self) -> None:
        self.a = 0
        self.b = 1

    def __iter__(self) -> Self:
        return self

    def __next__(self) -> int:
        ret = self.a
        self.a, self.b = self.b, self.a + self.b
        return ret


### Collatz Generator

An Generator function for Collatz sequences, which are bounded (for our purposes), terminating when the value reaches 1.



In [3]:
def collatz(n: int) -> Generator[int, None, None]:
    """Yield the Collatz sequence for the given input"""
    assert n > 0
    while n != 1:
        yield n
        n = 3 * n + 1 if n % 2 else n // 2
    yield 1

## Construction and extraction

Examples of constructing `Itr` objects from different data types and performing materialisation operations like
 `collect()`, `count()`, and `last()`.

Construct from a generator, `take` some values and `collect` into a set. Then get the `next_chunk`, and the `nth` after that:

NB `collect` returns a tuple by default but supports `list` `set` and `dict`:

In [4]:
f = Itr(Fibonacci())
display(f.take(10).collect(set))
display(f.next_chunk(10))
f.nth(10)  # 30th value (1-indexed)

{0, 1, 2, 3, 5, 8, 13, 21, 34}

(55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181)

514229

With a bounded generator we can just `collect` directly:

In [5]:
Itr(collatz(19)).collect()

(19, 58, 29, 88, 44, 22, 11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1)

Some points to note:
- an `nth` call that overruns the sequence raises a `StopIteration`
- a `skip` call that overruns the sequence raises a `StopIteration`
- `step_by` yields an item and then steps (same behaviour as `itetools.islice`)
- constructing `Itr` directly from a dictionary captures only the keys, use the `items` method to caputure key, 
value tuples.
- `iter` and `next`: `iter(itr)` is equivalent and interchangeable with `itr.__iter__()`. Same applies to `next(itr)` 
and `itr.next()` or `itr.__next__()`


In [6]:
with contextlib.suppress(StopIteration):
    Itr(collatz(4)).nth(10)  # Collatz(4) = 4,2,1

with contextlib.suppress(StopIteration):
    Itr(collatz(4)).skip(10)

display(Itr(collatz(19)).step_by(3).collect())

dict_it = Itr({"a": 1, "b": 2})
display(dict_it.collect())
dict_it = Itr({"a": 1, "b": 2}.items())
display(dict_it.collect())

(19, 88, 11, 52, 40, 5, 4)

('a', 'b')

(('a', 1), ('b', 2))

## Chaining

Comparing `Itr`'s ability to chain vs itertools. Note that in the second expression the generator is explicitly 
materialised into a `list` in order to reverse it - `Itr.rev` can't avoid this either, but it's done internally.

In [7]:
import itertools

Itr(collatz(19)).rev().step_by(3).skip(5).map(lambda x: x * x).filter(lambda x: x % 2 == 0).for_each(print)

# itertools equivalent
for item in filter(
    lambda x: x % 2 == 0,
    (x * x for x in itertools.islice(itertools.islice(reversed(list(collatz(19))), None, None, 3), 5, None)),
):
    print(item)

484
484


Using `inspect` allows intermediate steps to be logged. As there's a side-effect you might even want to discard the output, using `consume`:

In [8]:
Itr(collatz(19)).rev().step_by(3).inspect(print).skip(5).map(lambda x: x * x).filter(lambda x: x % 2 == 0).consume()

1
8
10
13
17
22
29



### Combining and Splitting Iterators

Methods for combining, splitting, or transforming multiple iterators.

`chain` concatenates two iterables:



In [9]:
Itr(collatz(5)).chain(collatz(6)).collect()

(5, 16, 8, 4, 2, 1, 6, 3, 10, 5, 16, 8, 4, 2, 1)

`zip` combines elements from iterables into tuples (stopping when either is exhausted):

In [10]:
Itr(collatz(5)).zip(collatz(6)).collect()

((5, 6), (16, 3), (8, 10), (4, 5), (2, 16), (1, 8))

`unzip` splits into two separate iterators: 
(NB The original iterator must yield pairs)

In [11]:
itr1, itr2 = Itr(collatz(5)).zip(collatz(6)).unzip()
itr1.collect(), itr2.collect()

((5, 16, 8, 4, 2, 1), (6, 3, 10, 5, 16, 8))

`intersperse` Inserts a value between items:

In [12]:
Itr(collatz(8)).intersperse("abcde").collect()

(8, 'abcde', 4, 'abcde', 2, 'abcde', 1)

`interleave` alternates values from two sequences (and ends when either sequence is exhausted):

In [13]:
Itr(collatz(5)).interleave(Itr(collatz(5)).rev()).collect()

(5, 1, 16, 2, 8, 4, 4, 8, 2, 16, 1, 5)


`enumerate` adds an index to each item:



In [14]:
Itr(collatz(5)).enumerate(start=1).collect()

((1, 5), (2, 16), (3, 8), (4, 4), (5, 2), (6, 1))

### Transformation and Filtering

Methods for transforming elements or filtering based on conditions.

`filter` keeps elements that satisfy a predicate.



In [15]:
Itr(collatz(19)).filter(lambda i: 4 <= i < 9).collect()

(5, 8, 4)

`map` transforms each element in a sequence and `for_each` passes each element to a function with side-effects (return values are not collected):


In [16]:
Itr(fibonacci()).take(5).map(lambda i: i**0.5).for_each(print)

0.0
1.0
1.0
1.4142135623730951
1.7320508075688772


`starmap` is a variant that expands each element as `*args` so that multi-argument functions can be called (each item must be a sequence):

In [17]:
def add(x, y):
    """Sum x and y"""
    return x + y


# 5 16 8 4 2 1
# 0  1 1 2 3 5
Itr(collatz(5)).zip(fibonacci()).starmap(add).collect()

(5, 17, 9, 6, 5, 6)


`flatten` and `flat_map`: both flatten a sequence of sequences, the latter also transforms the flattened items:



In [18]:
display(Itr(fibonacci()).take(6).map(lambda n: (n,) * n).flatten().collect())

display(Itr(fibonacci()).take(6).map(lambda n: (n,) * n).flat_map(lambda i: i * i).collect())

(1, 1, 2, 2, 3, 3, 3, 5, 5, 5, 5, 5)

(1, 1, 4, 4, 9, 9, 9, 25, 25, 25, 25, 25)

`partition` splits an iterator into two based on a predicate. The original iterator is consumed.

In [19]:
it0 = Itr(fibonacci()).take(20)
it1, it2 = it0.partition(lambda x: x % 2 == 0)
# it0 is consumed when it1 or it2 is accessed
it1.collect(), it2.collect(), it0.collect()

((0, 2, 8, 34, 144, 610, 2584),
 (1, 1, 3, 5, 13, 21, 55, 89, 233, 377, 987, 1597, 4181),
 ())

`copy` creates a shallow copy of the iterator's state:

In [20]:
it0 = Itr(collatz(10))
it1 = it0.copy()
next(it0)  # Advance it0 (only)
next(it1), next(it0)

(10, 5)

`batched` groups elements into tuples of a specified size: (The final item may be shorter.)

In [21]:
it = Itr(collatz(19)).batched(6)
it.collect()

((19, 58, 29, 88, 44, 22),
 (11, 34, 17, 52, 26, 13),
 (40, 20, 10, 5, 16, 8),
 (4, 2, 1))

`pairwise` and `rolling`: generate rolling windows of elements:

In [22]:
Itr(collatz(10)).rolling(3).collect()

((10, 5, 16), (5, 16, 8), (16, 8, 4), (8, 4, 2), (4, 2, 1))

`groupby` groups consecutive elements by a key. It's often convenient to `collect` into a `dict`:

In [23]:
Itr(collatz(19)).groupby(lambda n: n % 3).collect(dict)

{1: (19, 58, 88, 22, 34, 52, 13, 40, 10, 16, 4, 1),
 2: (29, 44, 11, 17, 26, 20, 5, 8, 2)}


### Aggregation

Methods for reducing an iterator to a single value:

- `fold`: Applies a function cumulatively to the elements, accumulating the results, starting with an initial value.
- `reduce`: is the same but uses the first element as the initial value.

In [24]:
Itr(collatz(10)).fold(0, add), Itr(collatz(10)).reduce(add)

(46, 46)


### Predicates and Terminal Operations

Methods that return a boolean or consume the iterator for side effects.

`all` checks if all elements satisfy a predicate; `any` check if at least one element satisfies the predicate:

Note that `all` will always consume the entire iterator (and should never be using on unbounded sequences), whereas 
`any` will stop when the predicate is satified

`max` and `min` consume elements and return the maximum and minimum elements respectively.

In [25]:
itr = Itr(collatz(8))
display(itr.all(lambda x: x < 10))
# itr.next()  # would raise StopIteration

itr = Itr(collatz(10))
display(itr.copy().collect())
display(itr.any(lambda x: x < 10))  # satisfied by second element
display(itr.next())  # yields third element

itr.max()  # now the fourth element, itr is now exhausted

True

(10, 5, 16, 8, 4, 2, 1)

True

16

8

### Iterator Manipulations

`skip_while` skips elements as long as the predicate is true. `take_while()` conversely will take elements until the 
predicate is no longer true:

In [26]:
display(Itr(collatz(19)).skip_while(lambda i: i < 80).collect())  # skips the first 3
Itr(collatz(19)).take_while(lambda i: i < 80).collect()  # takes the first 3

(88, 44, 22, 11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1)

(19, 58, 29)

`cycle` repeats the iterator elements indefinitely:

In [27]:
it = Itr(collatz(5)).cycle()
it.take(12).collect()

(5, 16, 8, 4, 2, 1, 5, 16, 8, 4, 2, 1)


`repeat` repeats the iterator `n` times:
    


In [28]:
Itr(collatz(5)).repeat(3).collect()

(5, 16, 8, 4, 2, 1, 5, 16, 8, 4, 2, 1, 5, 16, 8, 4, 2, 1)

`product` produces the Cartesian product with another iterable:

In [29]:
Itr(collatz(5)).product(collatz(2)).collect()

((5, 2),
 (5, 1),
 (16, 2),
 (16, 1),
 (8, 2),
 (8, 1),
 (4, 2),
 (4, 1),
 (2, 2),
 (2, 1),
 (1, 2),
 (1, 1))